In [59]:
import cv2
#CV2 loads everythings as BGR
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label
from tqdm import tqdm
import collections

test_image_list = glob.glob("./test_images/*.jpg")

In [102]:
#HOG Feature Extractor
def patch_analyzer(image, fd1=None, patch_loc=None, size=(16,16), nbins=16, bins_range=(0,256)):
    if fd1 is None and patch_loc is None:
        grey_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        fd, hog_image = hog(grey_img, orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(1, 1), visualise=True)
    else:
        y = patch_loc.startPosY
        ypos_end = patch_loc.endPosY
        x = patch_loc.startPosX
        xpos_end = patch_loc.endPosX
        #print(y,ypos_end, x, xpos_end)
        hog_image = fd1
        fd = fd1[y:ypos_end, x:xpos_end].ravel()


    #Color Space Extraction

    #conv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    conv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)

    #Downsample 0.25x from original
    #Original patch is 64px
    # Use cv2.resize().ravel() to create the feature vector
    scale_feature = cv2.resize(conv_img, size).ravel() 


    #Create Histograms from scaled image
    c1_hist = np.histogram(image[:,:,0], bins=nbins, range=bins_range)
    c2_hist = np.histogram(image[:,:,1], bins=nbins, range=bins_range)
    c3_hist = np.histogram(image[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((c1_hist[0], c2_hist[0], c3_hist[0]))


    #Combine and Normalized Data
    feature_list = [fd, scale_feature, hist_features]
    # Create an array stack, NOTE: StandardScaler() expects np.float64
    tmp_x = np.concatenate(feature_list).astype(np.float64)

    return tmp_x, image, hog_image

In [ ]:
#Train the classifier using patch_analyzer
vehicles = glob.glob('vehicles/*/*.png')
non_vehicles = glob.glob('non-vehicles/*/*.png')
image_loc_list = non_vehicles + vehicles

features_array = np.asarray([patch_analyzer(cv2.imread(image))[0] for image in tqdm(image_loc_list)])

np.save('LinearSVC_GTI_KTTT', features_array)


In [97]:
#Number of non-cars are represented by zeros
#Number of cars are represented by ones
img_labels = np.concatenate([np.zeros(8968), np.ones(8792)])

features_array = np.load('LinearSVC_GTI_KTTT.npy')

print(np.ndim(features_array))

X_train, X_test, y_train, y_test= train_test_split(features_array, img_labels, test_size=0.35)

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)

print("LinearSVC Reached")

#Implement Linear SVM Classifier
clf = LinearSVC(C=0.001,verbose=1, random_state=0)
clf.fit(scaled_X_train, y_train)

#Run trained classifier on test image
print()

print(clf.score(scaled_X_train, y_train)*100)

print(clf.score(scaled_X_test, y_test)*100)

2
LinearSVC Reached
[LibLinear]
99.4196119196
98.536036036


In [81]:
X_train = np.load('LinearSVC_GTI_KTTT.npy')
y_train = np.concatenate([np.zeros(8968), np.ones(8792)])

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
scaled_X_train = X_scaler.transform(X_train)

print("LinearSVC Reached")

#Implement Linear SVM Classifier
clf = LinearSVC(C=0.001,verbose=1, random_state=0)
clf.fit(scaled_X_train, y_train)

LinearSVC Reached
[LibLinear]

LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=1)

In [109]:
## Return a list of image patches
test_image = cv2.imread(test_image_list[0])

PatchImage = collections.namedtuple('PatchImage', ['image', 'startPosX', 'startPosY', 'endPosX', 'endPosY'])

def sliding_window(img, y_start_scale=0, y_stop_scale=100, patch_size=64, stride=16, scale=1):
    #Convert to float-32 value and normalize
    image = img.astype(np.float32)/255
    trans_img = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    imshape = trans_img.shape
    y_start = np.int(imshape[0]/scale*(y_start_scale/100))
    y_stop = np.int(imshape[0]/scale*(y_stop_scale/100))
    x_stop = np.int(imshape[1]/scale)
    if scale != 1:
        trans_img = cv2.resize(trans_img, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        y_stop = np.int(imshape[0]/scale)

    
    patch_list = []
    
    for x in range(0, x_stop-patch_size, stride):
        for y in range(y_start, y_stop-patch_size, stride):
            ypos_end = y+patch_size
            xpos_end = x+patch_size
            cur_patch = trans_img[y:ypos_end, x:xpos_end]
            #print((x, y), (xpos_end, ypos_end))
            # Extract image and locations for this patch
            patch_list.append(PatchImage(image=cur_patch, startPosX=x, startPosY=y, endPosX=xpos_end, endPosY=ypos_end))
            #patch_list.append([cur_patch, (x, y), (xpos_end, ypos_end)])
    return patch_list, trans_img

#Feed function trained LinearSVC classifier and list of patches to predict locations
def search_windows(img_list):
    positives_list = []
    for patch in img_list:
        features = patch_analyzer(image=patch.image, fd1=hog_image, patch_loc=patch)[0]
        prediction = clf.predict(X_scaler.transform(np.asarray(features)))
        if prediction == 1:
            positives_list.append([(patch.startPosX, patch.startPosY), (patch.endPosX, patch.endPosY)])
    return positives_list

# Here is your draw_boxes function from the previous exercise
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        # Coordinates expected in (x,y) and (x1,y1)
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        #print(bbox[1][0])
    # Return the image copy with boxes drawn
    return imcopy

#Iterate through three different scales
scale_list = [0.5, 1, 1.5, 2]

for scale in scale_list:
    grey_img = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    hfeat, hog_image = hog(grey_img, orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(1, 1), visualise=True)   
    search_list, HLS_image = sliding_window(test_image, y_start_scale=50, y_stop_scale=95, scale=scale)
    potential_cars = search_windows(search_list)
    plt.imshow(draw_boxes(potential_cars, HLS_image))
    print(scale)
    plt.show()

/home/jnee/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,1328)

In [ ]:
#Overlay outputs into heatmap, generate heatmap image + bounding box cluster images

def heat_map(img, box_list, threshold=0):
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    #Iterate through all the positively identified windows
    for entry in box_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        img[entry[0][1]:entry[1][1], entry[0][0]:box[1][0]] += 1
    #Zero out pixels below the threshold
    img[img <= threshold] = 0
    return img

#Labeled box code same as lesson function
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

#Draw bounding boxes around potential cars
draw_image = np.copy(test_image)

#Generate a list of boxes around positively identified cars
box_list = search_windows(patch_list)  

# Add heat to each box in box list and threshold the image for false positives
heat = heat_map(draw_image,box_list, 2)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
bounded_image = draw_labeled_bboxes(draw_image, labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(bounded_image)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heatmap, cmap='hot')
plt.title('Heat Map')
fig.tight_layout()

In [ ]:
#Open the input video and break it out into constituent frames
print(cv2.__version__)
vidcap = cv2.VideoCapture('')
success,image = vidcap.read()
count = 0
success = True
while success:
  success,image = vidcap.read()
  print 'Read a new frame: ', success
  box_list = search_windows(image)
  # Add heat to each box in box list and threshold the image for false positives
  heatmap = np.clip(heat_map(heat,box_list, 2), 0, 255)
  #Merge the pixels
  merged_cars = label(heatmap)
  draw_labeled_bboxes(merged_cars)
  cv2.imwrite("frame%d.png" % count, image)     # save frame as PNG file
  count += 1

In [ ]:
#Read all images with bounded boxes and compile into video
img1 = cv2.imread('1.jpg')
img2 = cv2.imread('2.jpg')
img3 = cv2.imread('3.jpg')

height , width , layers =  img1.shape

video = cv2.VideoWriter('video.avi',-1,1,(width,height))

video.write(img1)
video.write(img2)
video.write(img3)

cv2.destroyAllWindows()
video.release()